# KNIT7 One-way Latency Demo Part 1: Run OWL 

Script for running OWL inside a 3-node slice. Output for each measurement is saved as a pcap file on the destination node. 

Note:
- Current implementation on IPv4 experimenter interfaces only. 
- Assumes each node has only 1 experimental interface.

**Last tested: 2023/09/15 by MH**

# First things first

## Import fablib and mflib

- fablib version > 1.5.4
- mflib version > 1.0.3

In [ ]:
from fabrictestbed_extensions.fablib.fablib import fablib
import json

In [ ]:
from mflib import owl

## Identify the slice 

If necessary, create a slice with [knit7_demo_prep1](./knit7_owl_demo_prep1.ipynb)

In [ ]:
slice_name = "PTP_slice"

try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

In [ ]:
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

try:    
    node1 = slice.get_node(name=node1_name)
    node2 = slice.get_node(name=node2_name)
    node3 = slice.get_node(name=node3_name)
    nodes = slice.get_nodes()
except Exception as e:
    print(f"Fail: {e}")   

## Look up IPs of nodes and decide which IP address to use 

Checking experiment IP addresses by filtering out meas_network addresses. Asssumes one experimenter interface per node.

In [ ]:
node_ip_list = owl.nodes_ip_addrs(slice)
print(node_ip_list)

##  (Optional) Check OWL prerequisites on all nodes in the slice

Runs the following test and prints output on each node:

+ `git clone`
+ `ps -ef | grep phc2sys`
+ `docker --help`

In [ ]:
owl.check_owl_prerequisites(slice)

## Pull OWL image from DockerHub

In [ ]:
image_name="fabrictestbed/owl:0.1.7"

for node in nodes:
    node.execute("hostname")
    node.execute(f"sudo docker pull {image_name}") 

# Run the experiment

There are 3 ways to use OWL

1. Specify each link with sender-capturer pair: this should be easiest method for most users.
2. Start each sender and capturer manually: this is useful when you want to have more precise control
3. Start on all links: this starts OWL on every pair of nodes in the slice (excluding meas-node if there is one)


**Note: By default, `/home/rocky/owl-output` directory on each remote node will be mounted to the OWL container on each remote node, and all the output (pcap) files will be saved there. Moreover, the default behavior is set to overwrite the existing pcap file (from previous OWL runs) when a new OWL capturer session starts on that node. If this behavior is not desired, set `delete_previous_output` to `False`.**

## Method 1: Specify links by source and destination nodes
This examples runs OWL on the following links:

+ Node1 --> Node2
+ Node2 --> Node3
+ Node3 --> Node2

For each link, sender and capturer containers will start. If one node is a destination for more than 1 sender, only 1 capturer container instance will be created.

In [ ]:
owl.start_owl(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 120, delete_previous_output=True)

In [ ]:
owl.start_owl(slice, src_node=slice.get_node(name='Node2'), dst_node=slice.get_node(name='Node3'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 600, delete_previous_output=True)

In [ ]:
owl.start_owl(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 600, delete_previous_output=True)

### Check if they are running

In [ ]:
owl.check_owl_all(slice)

### Stop all OWL containers

It searches for running containers with the `owl` prefix. 

Note: If none, it will throw an error `"Docker container stop" requires 1 argument`, which just means there was no owl containers running on that node at that moment.

In [ ]:
owl.stop_owl_all(slice)

### Verify all OWL containers have been stopped and removed

In [ ]:
owl.check_owl_all(slice)

### Check if pcap files have been produced on the capturer (destination) nodes

By default, all the pcap files are saved in /home/rocky/owl-output as {dest_ip}.pcap on the desination node.

Note: If a node was not a capturer (detination), it will print `ls: cannot access '/home/rocky/owl-output': No such file or directory`

In [ ]:
owl_output_dir = '/home/rocky/owl-output'

for node in nodes:
    print(node.get_name())
    node.execute(f"ls -lh {owl_output_dir}")

## Method 2: start each sender, caputrer separately

If user desires more granular control, sender and capturer can be started/stopped independently.


In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node3'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_capturer(slice,  dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

### Check how the containers are running

In [ ]:
owl.check_owl_all(slice)

### Stop OWL containers on each node

The method finds all sender and capturer containers, stops and removes them.

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'))

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node3'))

In [ ]:
owl.stop_owl_capturer(slice,  dst_node=slice.get_node(name='Node2'))

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'))

### Verify that no OWL containers are left behind

In [ ]:
owl.check_owl_all(slice)

### Check if pcap files have been produced on the capturer (destination) nodes

By default, all the pcap files are saved in /home/rocky/owl-output as {dest_ip}.pcap on the desination node.

Note: If a node was not a capturer (detination), it will print `ls: cannot access '/home/rocky/owl-output': No such file or directory`

In [ ]:
owl_output_dir = '/home/rocky/owl-output'

for node in nodes:
    print(node.get_name())
    node.execute(f"ls -lh {owl_output_dir}")

## Method 3: start on all links

If OWL measurements are desired on ALL possible links, call the following method. It will start sender and capturer containers on all nodes with each node sending probe packets to all the rest of the nodes in the slice.

In [ ]:
owl.start_owl_all(slice, img_name=image_name, probe_freq=1, outfile=None, duration=600, delete_previous=True)

### Check if they are running

In [ ]:
owl.check_owl_all(slice)

### Stop all OWL containers

In [ ]:
owl.stop_owl_all(slice)

### Verify

In [ ]:
owl.check_owl_all(slice)

# (optional) Download the output (pcap) files

By default, all the pcap files are saved in `/home/rocky/owl-output` as `{dest_ip}.pcap` on the desination node.

## (optional)Check the availability of output files

In [ ]:
owl_output_dir = '/home/rocky/owl-output'

for node in nodes:
    print(node.get_name())
    node.execute(f"ls -lh {owl_output_dir}")

## (optional) Download the pcap files for analysis

In [ ]:
for node in nodes:
    
    print(f"Downloading pcap files from {node.get_name()}")
    
    local_output_dir = '/tmp/owl-output'
    owl.download_output(node, local_output_dir)

## (optional) Peek into the downloaded pcap files

The easiest way to convert the downloaded pcap file into a readable (ASCII) format is this command (on the terminal)`$ tcpdump -A -tt -n -l -r /path/to/pcap/file`

**Example**
```
$ tcpdump -A -tt -n -l -r /tmp/owl-output/Node1/10.140.130.2.pcap | tail -n 10
```

**output**
```
reading from file /tmp/owl-output/Node1/10.140.130.2.pcap, link-type EN10MB (Ethernet), snapshot length 262144
...
........ N.1694635151.320324053,435
1694635151.878668 IP 10.135.129.2.39749 > 10.140.130.2.5005: UDP, length 24
E..4.g@.=.>:
...
....E... ..1694635151.862285951,785
1694635152.331814 IP 10.136.3.2.39871 > 10.140.130.2.5005: UDP, length 24
E..4..@.=...
...
........ K.1694635152.320573973,436
```